# 스크래핑 복습 (part.3 basic 1 ~ basic 4)

In [ ]:
from bs4 import BeautifulSoup 
from urllib.request import urlopen 

In [ ]:
# 검색하고 싶은 단어 입력하기
word = 'happiness'

# 불러오려는 url 입력하기 
# 디폴트 url에 string 타입의 word 변수를 합쳐서 url 변수 생성
url = 'https://alldic.daum.net/search.do?q=' + word

# urlopen 함수를 통해 web 변수를 생성
web = urlopen(url)   # urlopen(url).read().decode('utf-8')

#print(web) #뭉텅이로 가져오게 됨.

# BeautifulSoup으로 web 페이지상의 HTML 구조를 파싱
web_page = BeautifulSoup(web, 'html.parser') #html 방식으로 해석해줘~ 라는 의미

print(web_page)

In [ ]:
box1 = web_page.find_all('span', {'class': 'txt_emph1'}) # 찾다
print(box1)

In [ ]:
box1[0].attrs

영화정보 출력하기

In [ ]:
url =  'https://movie.naver.com/movie/bi/mi/basic.naver?code=208077'
web = urlopen(url)
web_page = BeautifulSoup(web, 'html.parser')

In [ ]:
title = web_page.find('h3',{'class':'h_movie'}).find('a')
print(title.get_text())

In [ ]:
url = 'https://movie.naver.com/movie/bi/mi/detail.naver?code=208077' # 영화에 대한 배우/제작진 정보가 담긴 페이지
web = urlopen(url)
web_page = BeautifulSoup(web, 'html.parser')

In [ ]:
director = web_page.find('div', {'class':'dir_product'}).find('a', {'class':'k_name'}) 
print('Director:', end =" ")
print(director.get_text())

신문기사 출력하고 저장하기

In [ ]:
url = 'https://www.seattletimes.com/business/real-estate/zillows-zestimate-overvalued-a-washington-home-by-700-percent-in-a-case-of-algorithms-gone-wrong/'
web = urlopen(url)
source = BeautifulSoup(web, 'html.parser')
print(source)

In [ ]:
with open('seattletimes_text.txt', 'w', encoding = 'utf-8') as f:

    times = source.find('div', {'class': 'article-content'})
    article = times.find_all('p')
    for content in article:
        print(content.get_text())
        f.write(content.get_text()+ '\n')

문장 토큰화 하기

In [ ]:
import nltk

In [ ]:
# 전처리하고자 하는 문장을 String 변수로 저장한다
sentence = 'NLTK is a leading platform for building Python programs to work with human language data. It provides easy-to-use interfaces to over 50 corpora and lexical resources such as WordNet, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, wrappers for industrial-strength NLP libraries, and an active discussion forum.'

# 각 문장을 토큰화한 결과를 출력한다
nltk.word_tokenize(sentence)  # 문장을 토큰화해 출력한다 

In [ ]:
#영어 품사 태깅 하기 pos tagging
tokens = nltk.word_tokenize(sentence)
nltk.pos_tag(tokens)

In [ ]:
# nltk 모듈에서 Stopwords를 직접 불러온다
from nltk.corpus import stopwords

stopwords.fileids()


In [ ]:
# nltk 모듈에서 Stopwords를 직접 불러온다
from nltk.corpus import stopwords

# 영어의 stopwords를 불러와 변수에 저장한다
stopWords = stopwords.words('english') # 지원 언어 목록 : stopwords.fileids()

# How many stop words
print(len(stopWords))
print()

# stop words 출력
print(stopWords)

In [ ]:
stop_words = stopwords.words('english')
stop_words.append(',')
stop_words.append('.')

result = []

for token in tokens:
    if token.lower() not in stop_words:
        result.append(token)

print(result)

Lemmatization : 어근에 기반한 기본 사전형인 lemma 찾아줌

In [ ]:
lemmatizer = nltk.wordnet.WordNetLemmatizer()
print(lemmatizer.lemmatize("cats"))

In [ ]:
print(lemmatizer.lemmatize("swam", 'v'))

# news article scraping

In [ ]:
import requests # from urllib.request import urlopen
from bs4 import BeautifulSoup # HTTP Response -> HTML 

import pandas as pd
from datetime import datetime
import time 
import re

여러 뉴스 데이터 모으기

In [ ]:
https://search.naver.com/search.naver?where=news&query=홈트&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2018.01.01&de=2018.01.31&start=11

In [ ]:
def making_dates(year, month):
    if month < 10:
        ds = str(year) +".0" + str(month) + ".01"
        de = str(year) +".0" + str(month) + ".31" 
    else:
        ds = str(year) +"." + str(month) + ".01"
        de = str(year) +"." + str(month) + ".31"
    return ds, de
x1, x2 = making_dates(2018, 10)
x1

In [ ]:



#https://search.naver.com/search.naver?where=news&query=홈트&
#pd=3&
#ds=2018.01.01&
#de=2018.01.31&
#start=11

def yearly_df(year, query = '홈트'):

    month_counter = 1

    while month_counter < 13:

        ds, de = making_dates(year, month_counter)
        pages = ['01', '11', '21']

        for page in pages:
            url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='+query+'&sort=0&photo=0&field=0&pd=3&ds='+ds+'&de='+de+'&cluster_rank=15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&start='+page
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')

            # 1) 네이버 뉴스만 추려내기
            urls_list = []
            for urls in source.find_all('a', {'class': 'info'}):
                if urls.attrs['href'].startswith('https://news.naver.com/'):
                    urls_list.append(urls.attrs['href'])

            titles = []
            dates = []
            articles = []
            press_companies = []
            article_urls = []
            error_urls = []

            for url in urls_list:
                try:
                    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
                    web_news = requests.get(url, headers=headers).content
                    source_news = BeautifulSoup(web_news, 'html.parser')

                    # 2) 기사 제목
                    title = source_news.find('h3', {'id' : 'articleTitle'}).get_text()
                    titles.append(title)
                    print('Processing article : {}'.format(title))

                    # 3) 기사 날짜
                    date = source_news.find('span', {'class' : 't11'}).get_text()
                    dates.append(date)

                    # 4) 기사 본문
                    article = source_news.find('div', {'id' : 'articleBodyContents'}).get_text()
                    article = article.replace("\n", "")
                    article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                    article = article.replace("동영상 뉴스       ", "")
                    article = article.replace("동영상 뉴스", "")
                    article = article.strip()
                    articles.append(article)

                    # 5) 기사 URL 
                    article_urls.append(url)

                    # 6) 기사 발행 언론사
                    press_company = source_news.find('address', {'class' : 'address_cp'}).find('a').get_text()
                    press_companies.append(press_company)
                except:
                    print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
                    error_urls.append(url)
            
        month_counter += 1
            
    article_df = pd.DataFrame({'Title':titles,
                               'Date':dates, 
                               'Article':articles, 
                               'URL':article_urls, 
                               'PressCompany':press_companies})

        

    article_df.to_excel('result_{}.xlsx'.format(year), index=False, encoding='utf-8')
    article_df.head(10)
    
yearly_df(2018, '홈트')


# 워드클라우드 만들기

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib import font_manager, rc

from wordcloud import WordCloud
import re #정규표현식 사용

import nltk #text processing
from konlpy.tag import Okt #tag 안에 여러가지 형태소 분석기
from collections import Counter

%matplotlib inline

## 전체 코드

In [ ]:
# 데이터 불러오기
# '홈트레이닝' 연도별 뉴스 기사 하나의 df로 합치기
yrs = ['18','19','20','21']
df = pd.DataFrame()

for yr in yrs:
    temp = pd.read_excel('홈트레이닝news_20{}.xlsx'.format(yr))
    df = pd.concat([df, temp])

# df에 연도 범주형 변수 만들어주기
def date_to_year(date):
    return date[:4]

df['year'] = df['Date'].apply(lambda x: date_to_year(x))

# 연도별로 한 열의 시리즈를 리스트로 만들고 하나의 스트링으로 이어 붙이기
articles18 = df[df['year']=='2018']['Article'].tolist()
articles18 = ' '.join(articles18)
articles19 = df[df['year']=='2019']['Article'].tolist()
articles19 = ' '.join(articles19)
articles20 = df[df['year']=='2020']['Article'].tolist()
articles20 = ' '.join(articles20)
articles21 = df[df['year']=='2021']['Article'].tolist()
articles21 = ' '.join(articles21)


#==========================2018년만 일단..

## 단어 정규화 및 태깅 후 단어 등장 빈도 카운팅

# 단어 정규화 및 어근화 그리고 품사 태깅하기
tokenizer = Okt()
raw_pos_tagged = tokenizer.pos(articles18, norm = True, stem = True) #pos tagging
raw_pos_tagged

# 조사, 어미, 구두점, 외계어, SUFFIX, ALPHA, Determiner를 제외한 단어를 word_cleaned 리스트에 담기
del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', 
            '그렇다', '아니다', '이렇다', '그렇다', '어떻다',
            '으로', '에서', '하고', '보다', '관련', '따르다',
            '오다', '통해', '가다', '기자', '에는', '같다',
            '이라고', '까지'] 

word_cleaned = []

for word in raw_pos_tagged: # ('많다', 'Adjective')
    if not word[1] in ["josa", "Eomi", "Punctuation", "Foreign", "Suffix", "Alpha", "Determiner"]:
        if (len(word[0]) != 1) & (word[0] not in del_list):
            word_cleaned.append(word[0])

# word_cleaned에 담긴 단어들의 등장 횟수를 세는 dictionary 생성
# word_dic = {}
# for word in word_cleaned:
#     if word not in word_dic:
#         word_dic[word] = 1
#     else:
#         word_dic[word] += 1
# word_dic

# Counter를 통해서 단어들의 등장 횟수를 세는 dictionary 생성
result = Counter(word_cleaned)
word_dic = dict(result)

# 내림차순으로 가장 많이 나온 단어 순으로 정리
sorted_word_dic = sorted(word_dic.items(), key = lambda x: x[1], reverse = True)
sorted_word_dic

# 상위 50개만 살펴보고, 여기서 불용어 다시 추려보기
for word, count in sorted_word_dic[:50]:
    print("{0}({1})".format(word, count), end = " ")

In [ ]:
# 단어 등장 빈도 시각화 (nltk의 .Text()함수)
font_name = matplotlib.font_manager.FontProperties(fname="C:/Windows/Fonts/gulim.ttc").get_name() # NanumGothic.otf
matplotlib.rc('font', family=font_name)

word_counted = nltk.Text(word_cleaned) 
plt.figure(figsize=(15, 7)) # plot 영역(그래프 영역)의 크기를 지정합니다.
word_counted.plot(50) # "plot" the graph, 상위 50개 단어를 보여줍니다.

In [ ]:
# 막대 그래프 시각화
word_frequency = nltk.FreqDist(word_cleaned) # Frequency Distribution - dict 형태
df18 = pd.DataFrame(list(word_frequency.values()), word_frequency.keys())
result = df18.sort_values([0], ascending=False) # 빈도를 내림차순 정렬
result = result[:50] # 상위 50개만 시각화
result.plot(kind='bar', legend=False, figsize=(15,5))
plt.show()

## 연습

In [ ]:
df.head(5)

데이터 전처리단

In [ ]:
def date_to_year(date):
    return date[:4]

df['year'] = df['Date'].apply(lambda x: date_to_year(x))
df.head()

In [ ]:
# 연도별로 한 열의 시리즈를 리스트로 만들고 하나의 스트링으로 이어 붙이기
articles18 = df[df['year']=='2018']['Article'].tolist()
articles18 = ' '.join(articles18)
articles19 = df[df['year']=='2019']['Article'].tolist()
articles19 = ' '.join(articles19)
articles20 = df[df['year']=='2020']['Article'].tolist()
articles20 = ' '.join(articles20)
articles21 = df[df['year']=='2021']['Article'].tolist()
articles21 = ' '.join(articles21)

2018년만 가지고 해보기

In [ ]:
## 단어 정규화 및 태깅 후 단어 등장 빈도 카운팅

# 단어 정규화 및 어근화 그리고 품사 태깅하기
tokenizer = Okt()
raw_pos_tagged = tokenizer.pos(articles18, norm = True, stem = True) #pos tagging
raw_pos_tagged

# 조사, 어미, 구두점, 외계어, SUFFIX, ALPHA, Determiner를 제외한 단어를 word_cleaned 리스트에 담기
del_list = ['하다', '있다', '되다', '이다', '돼다', '않다', 
            '그렇다', '아니다', '이렇다', '그렇다', '어떻다',
            '으로', '에서', '하고', '보다', '관련', '따르다',
            '오다', '통해', '가다', '기자', '에는', '같다',
            '이라고', '까지'] 

word_cleaned = []

for word in raw_pos_tagged: # ('많다', 'Adjective')
    if not word[1] in ["josa", "Eomi", "Punctuation", "Foreign", "Suffix", "Alpha", "Determiner"]:
        if (len(word[0]) != 1) & (word[0] not in del_list):
            word_cleaned.append(word[0])

# word_cleaned에 담긴 단어들의 등장 횟수를 세는 dictionary 생성
# word_dic = {}
# for word in word_cleaned:
#     if word not in word_dic:
#         word_dic[word] = 1
#     else:
#         word_dic[word] += 1
# word_dic

# Counter를 통해서 단어들의 등장 횟수를 세는 dictionary 생성
from collections import Counter

result = Counter(word_cleaned)
word_dic = dict(result)
word_dic

In [ ]:
# 내림차순으로 가장 많이 나온 단어 순으로 정리
sorted_word_dic = sorted(word_dic.items(), key = lambda x: x[1], reverse = True)
sorted_word_dic

# 상위 50개만 살펴보고, 여기서 불용어 다시 추려보기
for word, count in sorted_word_dic[:50]:
    print("{0}({1})".format(word, count), end = " ")


# 에서 하고 보다 관련 따르다 오다 통해 가다 기자 에는 같다 이라고 까지

In [ ]:
# 단어 등장 빈도 시각화 (nltk의 .Text()함수)
font_name = matplotlib.font_manager.FontProperties(fname="C:/Windows/Fonts/gulim.ttc").get_name() # NanumGothic.otf
matplotlib.rc('font', family=font_name)

word_counted = nltk.Text(word_cleaned) 
plt.figure(figsize=(15, 7)) # plot 영역(그래프 영역)의 크기를 지정합니다.
word_counted.plot(50) # "plot" the graph, 상위 50개 단어를 보여줍니다.

In [ ]:
# 막대 그래프 시각화
word_frequency = nltk.FreqDist(word_cleaned) # Frequency Distribution - dict 형태
df18 = pd.DataFrame(list(word_frequency.values()), word_frequency.keys())
result = df18.sort_values([0], ascending=False) # 빈도를 내림차순 정렬
result = result[:50] # 상위 50개만 시각화
result.plot(kind='bar', legend=False, figsize=(15,5))
plt.show()

In [ ]:
# 워드 클라우드

# WordCloud 객체를 생성하기

# word_cloud = WordCloud().generate(text) : dict 가 아닌 줄글 텍스트 원문으로부터 워드클라우드를 만들 수 있습니다.
word_cloud = WordCloud(font_path="C:/Windows/Fonts/gulim.ttc",
                       width=2000, height=1000, # 이 부분을 수정하시면 실제 워드클라우드의 크기가 바뀝니다 (해상도가 바뀝니다)
                       # prefer_horizontal= 1.0, # 이 부분의 주석을 해제하시면 단어들이 가로로만 그려지게 됩니다. (0~1)
                       background_color='black')

word_cloud.generate_from_frequencies(word_dict)